- requests나 beautifulsoup을 사용해서 하면 네이버가 막음.

In [1]:
from fake_useragent import UserAgent
from selenium import webdriver
import time

In [2]:
categories = {
100 : '정치',
    101 : '경제',
    102 : '사회',
    103 : '생활/문화',
    104 : '세계',
    105 :'IT/과학'}

In [3]:
category, date, page = 100, 20210205, 1
url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}'.format(category, date, page)

In [4]:
driver = webdriver.Chrome()
driver.get(url)

In [5]:
articles = driver.find_elements_by_css_selector(".list_body > ul > li")
len(articles)

20

In [6]:
articles[0]

<selenium.webdriver.remote.webelement.WebElement (session="b9a30baa8d18689526c37c243f307dc4", element="6670e7cb-07f6-4114-957e-48ce5e813fd1")>

In [7]:
title = articles[0].find_element_by_css_selector('dt:not(.photo) > a').text
link = articles[0].find_element_by_css_selector('dt:not(.photo) > a').get_attribute('href')
title, link

('"대선후보 적합도…윤석열 28.3%·이재명 26.6%·이낙연 14.8%"',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=016&aid=0001790561')

- 제목이랑 링크만 확보하면 requests 씹가능
    - 이었는데 다시 막힘....
    - fake useragent 해보자
    - 아니면 selenium인데 너무느리네

In [18]:
headers = {'User-Agent':UserAgent().Chrome}
response = requests.get(link, headers=headers)
dom = BeautifulSoup(response.text, 'html5lib')

In [19]:
content = dom.select_one('#articleBodyContents').text.replace('\n', '').replace('\\','').replace('\t','').replace('{}','').replace('// flash','').replace('function _flash_removeCallback()','')
content

" 오류를 우회하기 위한 함수 추가 윤석열,이재명,이낙연[헤럴드경제] 차기 대선후보 적합도 조사에서 윤석열 검찰총장이 이재명 경기도지사를 오차범위 내 앞선다는 여론조사 결과가 5일 나왔다.조원씨앤아이가 시사저널 의뢰로 지난 2~3일 전국 만 18세 이상 남녀 1천11명을 대상으로 조사한 결과 윤 총장은 28.3%를 기록하며 이 지사(26.6%)를 1.7%포인트 앞섰다.더불어민주당 이낙연 대표는 14.8%로 집계됐다.무소속 홍준표 의원(8.0%), 정세균 국무총리(3.5%), 유시민 노무현재단 이사장(2.9%), 원희룡 제주도지사(2.6%)가 뒤를 이었다.윤 총장은 이 지사와의 가상 양자 대결에서도 42.0%를 기록하며 이 지사(39.5%)를 미세한 차이로 꺾었다. 윤 총장과 이 대표와의 가상 양자 대결에서 윤 총장은 40.5%를 기록하면서 이 대표(31.7%)를 오차범위 밖으로 앞섰다.다만 윤 총장의 정치 참여의 적절성 여부를 묻는 말에선 '적절하다'(46.6%)와 '부적절하다'(45.2%)가 팽팽했다.이 조사의 오차범위는 95% 신뢰수준에 ±3.1%포인트다.자세한 내용은 중앙선거여론조사심의위원회 홈페이지를 참조하면 된다.onlinenews@heraldcorp.com▶환경적 대화기구 '헤럴드에코'▶밀리터리 전문 콘텐츠 ‘헤밀’▶헤럴드경제 네이버 채널 구독- Copyrights ⓒ 헤럴드경제 & heraldbiz.com, 무단 전재 및 재배포 금지 -"

In [20]:
response = requests.get('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010333342', headers=headers)
dom = BeautifulSoup(response.content, 'html5lib')
dom.select_one('#articleBodyContents').text

'\n\t\n\t\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n\t\n\t[자료사진][서울=뉴시스]박미소 기자 = 황기철 국가보훈처장이 5일 설명절계기 정책현장방문의 일환으로 서울 서초구 서울남부보훈지청을 방문하여 직원들과 인사를 나누고 있다. (사진=국가보훈처 제공) 2021.02.05. photo@newsis.com 재판매 및 DB 금지[사진 영상 제보받습니다] 공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부(n-photo@newsis.com)로 보내주시면 적극 반영하겠습니다.<ⓒ 공감언론 뉴시스통신사. 무단전재-재배포 금지>\n\t\n\t'

In [12]:
def get_article(category, date, page):
    url = 'http://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}'.format(category, date, page)
    driver = webdriver.Chrome()
    driver.get(url)
    articles = driver.find_elements_by_css_selector(".list_body > ul > li")
    headers = {'User-Agent':UserAgent().Chrome}
    datas = []
    for article in articles:
        time.sleep(2)
        try: 
            title = article.find_element_by_css_selector('dt:not(.photo) > a').text
            link = article.find_element_by_css_selector('dt:not(.photo) > a').get_attribute('href')
            
            response = requests.get(link, headers=headers)
            dom = BeautifulSoup(response.content, 'html5lib')
            content = dom.select_one('#articleBodyContents').text.replace('\n', '').replace('\\','').replace('\t','').replace('{}','').replace('// flash','').replace('function _flash_removeCallback()','')
            datas.append({
                'title':title,
                'link':link,
                'content':content,
                'category':category
            })
        except:
            headers = {'User-Agent':UserAgent().Chrome}
            response = requests.get(link, headers=headers)
            dom = BeautifulSoup(response.content, 'html5lib')
            content = dom.select_one('#articleBodyContents').text.replace('\n', '').replace('\\','').replace('\t','').replace('{}','').replace('// flash','').replace('function _flash_removeCallback()','')
            datas.append({
                'title':title,
                'link':link,
                'content':content,
                'category':category
            })
    driver.quit()
    return pd.DataFrame(datas)            

In [13]:
get_article(100, 20210205, 1)

,title,link,content,category
0,"""대선후보 적합도…윤석열 28.3%·이재명 26.6%·이낙연 14.8%""",https://news.naver.com/main/read.nhn?mode=LSD&...,"오류를 우회하기 위한 함수 추가 윤석열,이재명,이낙연[헤럴드경제] 차기 대선후보 ...",100
1,"유시민 “‘86책임론’, 보수 언론이 지어낸 프레임”",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 /유튜브유시민 노무현재단 이사장은 이른바 ’86 ...,100
2,"대선후보 적합도…“윤석열(28.3%),이재명(26.6%) 앞서”",https://news.naver.com/main/read.nhn?mode=LSD&...,"오류를 우회하기 위한 함수 추가 조원씨앤아이, 시사저널 의뢰 조사차기 대선후보 적...",100
3,"권영세 ""동자동 쪽방촌 개발, 빈곤 탈출이 중요..복지 확대해야""",https://news.naver.com/main/read.nhn?mode=LSD&...,"오류를 우회하기 위한 함수 추가 ""선거 전부터 쪽방촌 골목 돌며어르신들 주거 환경...",100
4,"유시민 ""통일, 목표로 삼지 말아야…자유왕래가 우선""",https://news.naver.com/main/read.nhn?mode=LSD&...,"오류를 우회하기 위한 함수 추가 유시민 노무현재단 이사장이 남북통일과 관련해, ""...",100
5,“택시 기본요금 1200원” 답했다 진땀 뺀 변창흠,https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 변창흠 국토교통부 장관이 5일 국회 본회의에서 ...,100
6,"봉하 간 추미애 “공수처출범, 이제 盧께 제대로 인사”",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 페이스북에 故노무현 전 대통령 묘소 방문 사진 올...,100
7,추미애 “촛불 힘으로 공수처 출범”…盧 전 대통령 묘소 참배,https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 봉하 찾은 추미애 전 장관 - 추미애 전 법무부 ...,100
8,"""대선후보 적합도…윤석열 28.3%·이재명 26.6%·이낙연 14.8%""",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 [연합뉴스TV 제공](서울=연합뉴스) 강민경 기자...,100
9,쇠사슬·인간벽 뒤에 숨은 김명수 “잘해보겠다” 사퇴 거부,https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 野의원들 대법원 항의 방문하자 출입문 봉쇄하고 버...,100


In [6]:
date = 20210204

In [9]:
dfs = []
for category in categories:
    print(categories[category],end=' ')
    for page in range(1, 6):
        print(page, end=' ')
        df = get_article(category, date, page)
        dfs.append(df)
        
articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

정치 1 2 3 4 5 경제 1 2 3 4 5 사회 1 2 3 4 5 생활/문화 1 2 3 4 5 세계 1 2 3 4 5 IT/과학 1 2 3 4 5 

,title,link,content,category
595,"페북, 코로나19·백신 관련 허위정보 1천만개 넘게 지웠다",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 삭제·감소·알림 전략 통해 유해콘텐츠 문제 해결(...,105
596,"델타·아메리칸항공도 적자인데...대한항공, 코로나 뚫고 '흑자'",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 대한항공이 코로나 19에 따른 항공업 고사 위기...,105
597,"CJ ENM, 작년 영업익 2721억원…4Q TV 광고 회복",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 코로나 여파 커머스 영화 부문 실적 주춤(지디넷코...,105
598,"하이퍼커넥트, 지난해 韓 앱 배급사 중 매출 6위",https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 앱애니 '2021 상위 퍼블리셔 어워드'에 2개 ...,105
599,新 폴더블폰 내놓는 화웨이…고가 스마트폰 '매각설' 잠재울까,https://news.naver.com/main/read.nhn?mode=LSD&...,오류를 우회하기 위한 함수 추가 첫 인폴딩 방식 적용한 '메이트 X2' 22일 발...,105


In [11]:
articles_df.to_csv('articles_df.csv')

In [24]:
articles_df = pd.read_csv('articles_df.csv')

In [22]:
articles_df['content'] = articles_df['content'].str.replace('오류를 우회하기 위한 함수 추가', '')

In [17]:
articles_df.to_csv('articles_df.csv')

In [25]:
pd.read_csv('articles_df.csv')

,Unnamed: 0,Unnamed: 0.1,title,link,content,category
0,0,0,‘재판 독립 침해’ 판사 탄핵 소추안 첫 가결,https://news.naver.com/main/read.nhn?mode=LSD&...,‘재판 독립 침해’ 판사 탄핵 소추안 첫 가결김지숙 (vox@kbs.co.kr)...,100
1,1,1,"변창흠, 뉴스데스크 출연…""도심에 신속 주택 공급해 국민 불안 해소할 것""",https://news.naver.com/main/read.nhn?mode=LSD&...,"변창흠 국토교통부 장관은 ""공공이 주도하는 개발사업을 통해 도심에 신속히 주택을...",100
2,2,2,文 “한반도 비핵화·항구적 평화 공동 노력”…바이든 “양국 70여 년간 계속 진전”,https://news.naver.com/main/read.nhn?mode=LSD&...,"文대통령, 32분 동안 바이든과 첫 한미정상 통화 / 양 정상 “조속히 포괄적 ...",100
3,3,3,25번째 대책 발표날… 이재명 “공직자가 다주택자면 정책 신뢰도 떨어져”,https://news.naver.com/main/read.nhn?mode=LSD&...,이재명 경기도지사. 경기사진공동취재단 정부가 25번째 부동산 정책...,100
4,4,4,"[종합] ""부산 약화시킬 것"" vs ""친일 관계 없어""…'한일 해저터널' 與野 설전",https://news.naver.com/main/read.nhn?mode=LSD&...,"野 부산 '한일 해저터널' 두고 공방與 ""일본 대륙진출 야심에 이용될 것"" 맹공...",100
...,...,...,...,...,...,...
595,595,595,"페북, 코로나19·백신 관련 허위정보 1천만개 넘게 지웠다",https://news.naver.com/main/read.nhn?mode=LSD&...,삭제·감소·알림 전략 통해 유해콘텐츠 문제 해결(지디넷코리아=안희정 기자)최근...,105
596,596,596,"델타·아메리칸항공도 적자인데...대한항공, 코로나 뚫고 '흑자'",https://news.naver.com/main/read.nhn?mode=LSD&...,대한항공이 코로나 19에 따른 항공업 고사 위기 속에서도 화물 운송 부문의 선...,105
597,597,597,"CJ ENM, 작년 영업익 2721억원…4Q TV 광고 회복",https://news.naver.com/main/read.nhn?mode=LSD&...,코로나 여파 커머스 영화 부문 실적 주춤(지디넷코리아=박수형 기자)CJ ENM...,105
598,598,598,"하이퍼커넥트, 지난해 韓 앱 배급사 중 매출 6위",https://news.naver.com/main/read.nhn?mode=LSD&...,앱애니 '2021 상위 퍼블리셔 어워드'에 2개 부문 톱10 올라[아이뉴스24 ...,105


- selenium으로 하는 방식

In [ ]:
driver.get(link)

In [ ]:
content = driver.find_element_by_css_selector('#articleBodyContents').text.replace('\n', '')
content

In [ ]:
def get_article(category, date, page):
    url = 'http://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}'.format(category, date, page)
    driver = webdriver.Chrome()
    driver.get(url)
    articles = driver.find_elements_by_css_selector(".list_body > ul > li")
    datas = []
    for article in articles:
        title = article.find_element_by_css_selector('dt:not(.photo) > a').text
        link = article.find_element_by_css_selector('dt:not(.photo) > a').get_attribute('href')
        driver = webdriver.Chrome()
        driver.get(link)
        content = driver.find_element_by_css_selector('#articleBodyContents').text.replace('\n', '').replace('\\','')
        driver.quit()
        datas.append({
            'title':title,
            'link':link,
            'content':content,
            'category':category,
        })
    driver.implicitly_wait(3)
    driver.quit()
    return pd.DataFrame(datas)

In [ ]:
get_article(100, 20210205,1)

In [ ]:
date =20210205

In [ ]:
dfs = []
for category in categories:
    print(categories[category], end = " ")
    for page in range(1, 6):
        print(page, end=" ")
        df = get_article(category, date, page)
        dfs.append(df)
articles_df = pd.concat(dfs, ignore_index=True)  # index reset해주는 거임
articles_df.tail()

# Multinomail NB

In [28]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [34]:
article_df = pd.read_csv('./articles_df.csv', usecols = ['title','link','content','category'])
article_df = articles_df[['category','title','link','content']]
article_df.tail()

,category,title,link,content
595,105,"페북, 코로나19·백신 관련 허위정보 1천만개 넘게 지웠다",https://news.naver.com/main/read.nhn?mode=LSD&...,삭제·감소·알림 전략 통해 유해콘텐츠 문제 해결(지디넷코리아=안희정 기자)최근...
596,105,"델타·아메리칸항공도 적자인데...대한항공, 코로나 뚫고 '흑자'",https://news.naver.com/main/read.nhn?mode=LSD&...,대한항공이 코로나 19에 따른 항공업 고사 위기 속에서도 화물 운송 부문의 선...
597,105,"CJ ENM, 작년 영업익 2721억원…4Q TV 광고 회복",https://news.naver.com/main/read.nhn?mode=LSD&...,코로나 여파 커머스 영화 부문 실적 주춤(지디넷코리아=박수형 기자)CJ ENM...
598,105,"하이퍼커넥트, 지난해 韓 앱 배급사 중 매출 6위",https://news.naver.com/main/read.nhn?mode=LSD&...,앱애니 '2021 상위 퍼블리셔 어워드'에 2개 부문 톱10 올라[아이뉴스24 ...
599,105,新 폴더블폰 내놓는 화웨이…고가 스마트폰 '매각설' 잠재울까,https://news.naver.com/main/read.nhn?mode=LSD&...,"첫 인폴딩 방식 적용한 '메이트 X2' 22일 발표…업계, 몸값 띄우기용 관측도..."


## 데이터 전처리 : 데이터 셋 분리

In [35]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size= 0.1, random_state=1)

## 모델 만들기

In [55]:
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf',MultinomialNB(alpha=0.1)), #Additive (Laplace/Lidstone) smoothing parameter
])

## 학습

In [56]:
model = clf.fit(X_train, y_train)

## 성능 평가

In [57]:
y_pred = model.predict(X_test)

In [58]:
y_test[:5], y_pred[:5]

(446    104
 404    104
 509    105
 455    104
 201    102
 Name: category, dtype: int64,
 array([104, 104, 105, 104, 102], dtype=int64))

In [59]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.89      0.80      0.84        10
         101       0.67      0.75      0.71         8
         102       0.62      0.50      0.56        10
         103       0.50      0.43      0.46         7
         104       0.94      0.94      0.94        16
         105       0.67      0.89      0.76         9

    accuracy                           0.75        60
   macro avg       0.71      0.72      0.71        60
weighted avg       0.75      0.75      0.75        60



## 모델 사용

In [60]:
categories ={
    100 : '정치',
    101 : '경제',
    102 : '사회',
    103 : '생활/문화',
    104 : '세계',
    105 : 'IT/과학'
}

In [64]:
contents = [
    '네이버와 카카오는 드론 기술 발전에 주력',
    '광화문 대한애국당 집회, 격렬 저항 속 강제 해산',
    '최근 환율 주가 예측 어려워, 증권사 골머리',
    'FTSE의 공개적인 공매도 금지 비판, 금융당국 골머리'
]

data = {
    'content' : contents,
    'category_code' : model.predict(contents)
}

In [65]:
df = pd.DataFrame(data)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주력,105
1,"광화문 대한애국당 집회, 격렬 저항 속 강제 해산",100
2,"최근 환율 주가 예측 어려워, 증권사 골머리",101
3,"FTSE의 공개적인 공매도 금지 비판, 금융당국 골머리",101


In [66]:
df['category'] = df['category_code'].map(lambda data: categories[data])
df

,content,category_code,category
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/과학
1,"광화문 대한애국당 집회, 격렬 저항 속 강제 해산",100,정치
2,"최근 환율 주가 예측 어려워, 증권사 골머리",101,경제
3,"FTSE의 공개적인 공매도 금지 비판, 금융당국 골머리",101,경제


In [68]:
df['proba'] = df['content'].apply(lambda data : model.predict_proba([data]))
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/과학,"[[0.22212297654109833, 0.1280571554038728, 0.1..."
1,"광화문 대한애국당 집회, 격렬 저항 속 강제 해산",100,정치,"[[0.2068209032972136, 0.1421666800984579, 0.14..."
2,"최근 환율 주가 예측 어려워, 증권사 골머리",101,경제,"[[0.09367028263001075, 0.4343242177074589, 0.1..."
3,"FTSE의 공개적인 공매도 금지 비판, 금융당국 골머리",101,경제,"[[0.24822997830911975, 0.28364383144587446, 0...."


# 모델 저장

In [70]:
pickle.dump(model, open('clf.pkl', 'wb'))